## Learning Objectives

### 1. Python Lists vs. NumPy Arrays
- Understand the structural and performance differences between Python lists and NumPy arrays.

### 2. Memory Structure of NumPy Arrays
- Learn how NumPy arrays store homogeneous data in contiguous memory, enabling efficient numerical computation.

### 3. Basic NumPy Array Creation and Attributes
- `np.array`
- `np.zeros`
- `np.arange`


In [1]:
import numpy as np

### 1. Python Lists vs. NumPy Arrays

In [3]:
arr_1d = np.array([1, 2, 3, 4, 5])
arr_2d = np.array([[1, 2, 3, 4, 5],
                  [6, 7, 8, 9, 10],
                  [11, 12, 13, 14, 15],
                  [16, 17, 18, 19, 20],
                  [21, 22, 23, 24, 25]])

In [5]:
arr_2d1 = np.arange(26, 51).reshape(5, 5)

In [6]:
print("1D Array:", arr_1d)
print("Shape:", arr_1d.shape)
print("Dtype:", arr_1d.dtype)

1D Array: [1 2 3 4 5]
Shape: (5,)
Dtype: int64


In [7]:
print("\n2D Array:\n", arr_2d)
print("Shape:", arr_2d.shape)
print("Dtype:", arr_2d.dtype)


2D Array:
 [[ 1  2  3  4  5]
 [ 6  7  8  9 10]
 [11 12 13 14 15]
 [16 17 18 19 20]
 [21 22 23 24 25]]
Shape: (5, 5)
Dtype: int64


In [8]:
print("\n2D Array:\n", arr_2d1)
print("Shape:", arr_2d1.shape)
print("Dtype:", arr_2d1.dtype)


2D Array:
 [[26 27 28 29 30]
 [31 32 33 34 35]
 [36 37 38 39 40]
 [41 42 43 44 45]
 [46 47 48 49 50]]
Shape: (5, 5)
Dtype: int64


### 2. Memory Structure of NumPy Arrays

In [ ]:
# 1 ms = 10e-3 s
python_list = list(range(1, 1000001))
%timeit sum(python_list)

20.4 ms ± 1.93 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
# 1 μs = 10e-6 s
numpy_list = np.arange(1, 1000001)
%timeit np.sum(numpy_list)

708 μs ± 65.7 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### 3. Basic NumPy Array Creation and Attributes

In [13]:
zeros = np.zeros((3, 4))
print(zeros)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [15]:
ones = np.ones((2, 3))
print(ones)

[[1. 1. 1.]
 [1. 1. 1.]]


In [18]:
range_arr = np.arange(0, 12, 2)
print(range_arr)
print(range_arr.reshape(2, 3))

[ 0  2  4  6  8 10]
[[ 0  2  4]
 [ 6  8 10]]


In [19]:
linspace_arr = np.linspace(0, 7, 5)
print(linspace_arr)

[0.   1.75 3.5  5.25 7.  ]


# Numpy Drill (10 problems)

In [20]:
import numpy as np
np.random.seed(0)

In [25]:
# Problem 1. 차원 인식
data = np.random.rand(365, 73, 144) # (time, lat, lon)

# 1. 이 배열에서 global mean time series의 shape는? 코드로 작성하라.
# (365, ) -> Numpy에서 shape는 항상 tuple로 표현하니까.
data.mean(axis=(1, 2)).shape

(365,)

### axis (a, b, c) <--> axis : 0, 1, 2
- 즉 axis=(1, 2)는 latitude, longitude에 대해 평균을 내고, time(axis=0)만 남기라는 뜻.

- 만약 data.mean(axis=0) 을 했다면, mean spatial field가 나온다.

- 남은 축 : `time`
- 축 개수 : 1개
- 길이 365
- -> 1차원 배열

In [26]:
# Problem 2 : axis 실수 방지하기!
# zonal_mean의 shape는 무엇인가? 이 연산의 물리적인 의미는 무엇이겠는가?
zonal_mean = data.mean(axis=2)

1. zonal_mean shape는 (time, lat) = (365, 73)
2. 시간에 따른 longitude 방향 평균 -> zonal mean

In [27]:
# Problem 3 : broadcasting의 함정
lat = np.linspace(-90, 90, 73)
weighted = data * np.cos(np.deg2rad(lat))

# 이 코드가 에러 없이 실행되는가? 그 이유는?

ValueError: operands could not be broadcast together with shapes (365,73,144) (73,) 

- data 는 (365, 73, 144), np.cos(np.deg2rad(lat))는 (73, )이다.
- numpy는 오른쪽 축부터 맞추려고 하다보니 (..., 73)으로 취급되어 마지막 축에 붙음. 그래서 broadcast가 안되는 것.

따라서 
p.cos(np.deg2rad(lat))[None, :, None]으로 두어야, broadcast가 성립된다.

### 즉, 축을 정확히 알고 있어야 간결해진다는 뜻이다.

In [29]:
weighted = data * np.cos(np.deg2rad(lat))[None, :, None]
print(weighted.shape)


(365, 73, 144)


In [32]:
## 문제 4 — 올바른 area-weighted mean 계산하기.
# latitude 가중 평균을 계산하는 올바른 코드는? -> shape: (365,)
weights = np.cos(np.deg2rad(lat))
w = weights[None, :, None]
# 합 / 기준값

result_latitude_weighted_sum = \
    np.sum(data * w, axis = (1, 2)) / np.sum(w)

In [31]:
print(result_latitude_weighted_sum.shape)

(365,)


In [33]:
result_latitude_weighted_sum

array([71.81034687, 71.98772365, 71.90226358, 71.78044686, 72.69218387,
       71.9072376 , 72.31076211, 72.09107801, 71.46433059, 72.34159221,
       71.79615319, 72.58981971, 71.54854715, 72.82350466, 71.55468085,
       72.31489909, 71.53695459, 72.31758665, 71.52525165, 72.45336663,
       71.77799914, 72.29639552, 71.42887058, 72.82422642, 71.98056171,
       71.9404165 , 73.23090058, 72.25043248, 71.87677564, 71.86656137,
       71.57403722, 71.63759514, 72.30919544, 72.61221837, 72.36169652,
       71.84270505, 72.17725293, 72.35821775, 71.59719138, 72.06535967,
       72.44964854, 72.1088863 , 71.91700286, 71.35270301, 71.85938703,
       72.01362052, 71.52200779, 71.9368533 , 72.0221785 , 71.88218053,
       72.15207597, 72.08701904, 71.91671046, 71.87895284, 72.14399283,
       72.02104133, 71.73022288, 73.15410186, 71.8063457 , 72.09947037,
       71.63406463, 71.75118868, 71.65566856, 72.23594202, 72.80323764,
       71.60584541, 70.9662686 , 72.69636261, 71.27107884, 72.92

In [36]:
# Problem 5 — view vs copy 
# `a`의 최종 값은? 왜 이런 일이 발생했는가? 이를 방지하려면 어떻게 해야 하는가?
a = np.arange(10) 
print(a)
b = a[2:8]
b[:] = -1
print(a)


[0 1 2 3 4 5 6 7 8 9]
[ 0  1 -1 -1 -1 -1 -1 -1  8  9]


In [40]:
# a = np.arange(10) -> [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
# b = a[2:8]  ->  [0, 1 list(b), 8, 9]
# b[:] = -1 -> list(b) = [-1, -1, -1 , -1, -1, -1]  
# -> a의 해당 메모리를 직접 수정한 것.
# print(a)  ->  [ 0  1 -1 -1 -1 -1 -1 -1  8  9]
# Numpy slicing은 기본적으로 view를 반환함.

a = np.arange(10)
print(a)
b = a[2:8].copy()
b[:] = -1
print(a)

[0 1 2 3 4 5 6 7 8 9]
[0 1 2 3 4 5 6 7 8 9]


In [41]:
## Problem 6. reshape 이후 데이터 오염
# `x`의 값은 변했는가?  이 동작이 위험한 이유는?

x = np.arange(12)
y = x.reshape(3, 4)
y[0, :] = 999

- 반드시 변하지.
- y가 x에 종속 된 상태인데 y를 변화시키면 x도..
- copy 쓰자

In [44]:
a1 = np.arange(60)
b1 = a1.reshape(3, 4, 5).copy()

b1[0, :, :] = 999

print(a1, '\n', b1)


[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59] 
 [[[999 999 999 999 999]
  [999 999 999 999 999]
  [999 999 999 999 999]
  [999 999 999 999 999]]

 [[ 20  21  22  23  24]
  [ 25  26  27  28  29]
  [ 30  31  32  33  34]
  [ 35  36  37  38  39]]

 [[ 40  41  42  43  44]
  [ 45  46  47  48  49]
  [ 50  51  52  53  54]
  [ 55  56  57  58  59]]]


In [45]:
# Problem 7 — NaN + dtype 함정
# 무슨 일이 발생하는가? 실전에서 가장 안전한 초기화 방식은?
arr = np.array([1, 2, 3, 4])
arr[2] = np.nan
arr

ValueError: cannot convert float NaN to integer

- NaN = floating-point 전용

- 가장 안전한 초기화 방식ㅇ란, NaN이 나중에 들어올 수도 있다는 가능성 자체를 전제를 두고 코드를 짜야한다는 의미이다.

따라서,

- `arr = np.full(4, np.nan)`
- `arr = np.full((365, 73, 144), np.nan)`
- `arr = np.empty(4, dtype=float),  arr[:] = np.nan` 처럼 메모리 할당을 한다.

In [47]:
arr1 = np.array([1, 2, 3, 4, 5], dtype = float)
arr1[2] = np.nan
arr1

array([ 1.,  2., nan,  4.,  5.])

In [48]:
arr2 = np.empty(5, dtype=float)
arr2[:] = np.nan
arr2

array([nan, nan, nan, nan, nan])

In [52]:
# Problem 8 — NaN 포함 평균 내보기
# np.mean(temp) 결과는? NaN을 무시한 평균은 어떻게 계산할 것인가?

temp = np.array([280., 282., np.nan, 285.])

# 결과는 nan으로 처리된다. (nan 값을 어떻게 계산하라는건지 알 수가 없으니까.)
print(np.mean(temp))

nan


In [63]:
# pandas : temp.isna()
# numpy  : np.isnan(temp)

temp1 = temp.copy()
temp1_data = temp1[~np.isnan(temp)]
float(np.mean(temp1_data))

282.3333333333333

In [64]:
np.nanmean(temp)

np.float64(282.3333333333333)

In [70]:
# Problem 9 : 성능 사고
# 이 코드는 왜 비효율적일까요? Numpy 한 줄로 대체해봅시다.
data = np.random.rand(365, 73, 144) # (time, lat, lon)

result = []
for t in range(365):
    result.append(data[t].mean())
result = np.array(result) # [] = (365, )
print(result.shape)

(365,)


In [ ]:
# data 내에서 한 번에 계산하지 않고 with 벡터화, loop를 활용하기 때문입니다.
# numpy는 한 번에 큰 배열을 처리할 수 있기 때문.
result2 = np.mean(data, axis=(1, 2))
print(result2.shape)

(365,)


In [73]:
# Problem 10 — 종합

# data: (time, lat, lon)
# 목표: latitude 가중 global mean time series
# 필요한 개념 3가지를 나열하라, 최종 코드를 작성하라 (2~3줄 이내)

# data = np.random.rand(365, 73, 144)
# 개념 1 linspace..?
lat = np.linspace(-90, 90, 73)
w = np.cos(np.deg2rad(lat))[None, :, None] # broadcasting
global_mean = np.sum(data * w, axis=(1, 2)) / np.sum(w)


In [74]:
print(global_mean.shape)

(365,)
